In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from PIL import Image
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [ ]:
labels=pd.read_csv(r"C:\Users\carol\Desktop\Adv PM\train-20181125T003601Z-001\labels.csv")
target={}
for i in labels['id']:
    target[i]=labels[labels['id']==i]['breed'].iloc[0]

In [ ]:
def load_dataset(path):
    #Load text files with categories as subfolder names.
    images = [f for f in os.listdir(path) if os.path.splitext(f)[-1] == '.jpg']
    dog_files=[]
    dog_targets=[]
    for image in images:
        file_id=image.rstrip('.jpg')
        dog_files.append(np.array(Image.open(image)))
        dog_targets.append(target[file_id])
    return dog_files, dog_targets

In [ ]:
def load_dataset_test(path):
    #Load text files with categories as subfolder names.
    images = [f for f in os.listdir(path) if os.path.splitext(f)[-1] == '.jpg']
    dog_files=[]
    dog_targets=[]
    for image in images:
        file_id=image.rstrip('.jpg')
        dog_files.append(np.array(Image.open(image)))
        #dog_targets.append(target[file_id])
    return dog_files#, dog_targets

In [ ]:
path2=r"C:\Users\carol\Desktop\Adv PM\train-20181125T003601Z-001\train"
path3=r"C:\Users\carol\Desktop\Adv PM\test-20181125T004858Z-001\test"

In [ ]:
os.chdir(path2)
train_files, train_targets = load_dataset(path2)

In [ ]:
os.chdir(path3)
test_files = load_dataset_test(path3)

In [ ]:
np.random.seed(seed=1)
train_len=len(train_files)
indices = np.random.permutation(train_len)
train_idx, val_idx = indices[:round(train_len*0.9)], indices[round(train_len*0.9):]

In [ ]:
temp2=Series(train_targets)
y_train, y_val = temp2[train_idx], temp2[val_idx]
y_train=list(y_train)
y_val=list(y_val)

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUCATED_IMAGED=True

In [ ]:
imgs = [f for f in os.listdir(path2) if os.path.splitext(f)[-1] == '.jpg']
img_pth=[]
for i in imgs:
    img_pth.append(join(path2,i))

In [ ]:
temp_img=Series(img_pth)
x_train_img, x_val_img = temp_img[train_idx], temp_img[val_idx]

In [ ]:
img_pth_tr=list(x_train_img)
img_pth_vl=list(x_val_img)

In [ ]:
imgs_te = [f for f in os.listdir(path3) if os.path.splitext(f)[-1] == '.jpg']
img_pth_te=[]
for i in imgs_te:
    img_pth_te.append(join(path3,i))

In [ ]:
valid_tensors = paths_to_tensor(img_pth_vl).astype('float32')/255
train_tensors = paths_to_tensor(img_pth_tr).astype('float32')/255
test_tensors = paths_to_tensor(img_pth_te).astype('float32')/255

In [ ]:
input_size=224
num_classes=120
S=1
train_len=len(train_files)

In [ ]:
inception_bottleneck=inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')
train_i_bf = inception_bottleneck.predict(train_tensors, batch_size=32, verbose=1)
valid_i_bf = inception_bottleneck.predict(valid_tensors, batch_size=32, verbose=1)
print('InceptionV3 train bottleneck features shape: {} size: {:,}'.format(train_i_bf.shape, train_i_bf.size))
print('InceptionV3 valid bottleneck features shape: {} size: {:,}'.format(valid_i_bf.shape, valid_i_bf.size))

In [ ]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_i_bf, (ytr * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_i_bf)
valid_preds = logreg.predict(valid_i_bf)